Daniel Aguilar
Homework3
CSP 228
Prof. James Liporace
04/14/2025

**Instructions**  
Progrmming Projects

Chapter 6 

6.2,6.3,6.4

Chapter 7 

7.1,.7.2,7.3,.7.4



In [15]:
# %%
"""
Problem 6.2:
Every positive integer can be divided into a set of integer factors.
The minimum set of factors must be a collection of prime numbers,
where a prime number is one that is only evenly divisible by 1 and itself.

Write a recursive function, factor(), that returns the list of integer
factors of x.


1. Only test factors between 2 and the square root of x (use lowest up to sqrt(x)).
2. Add an optional parameter to factor(x, lowest) for the lowest possible integer factor.
3. The recursive function should:
   a. Check base cases (0, 1, and negatives).
   b. If lowest evenly divides x (x % lowest == 0):
        - Add lowest to the factors of x divided by lowest.
   c. If lowest does NOT evenly divide x:
        - Look for factors with the next higher possible factor (lowest+1).
   d. A factor can appear more than once in the final list.
4. For negative x, return the factors of the positive version but with factor 1 replaced by -1.
5. Test on:
   -- Compound (nonprime) numbers
   -- Prime numbers
   -- Special cases of 0 and 1
   -- Negative integers

Use Chapter 5's LinkedList structure (plus __iter__ to traverse it).
"""

class Link:
    #One datum in a linked list.
    def __init__(self, data, next=None):
        self.__data = data
        self.__next = next

    def getData(self):
        return self.__data

    def getNext(self):
        return self.__next

    def setNext(self, link):
        if link is None or isinstance(link, Link):
            self.__next = link
        else:
            raise Exception("Next link must be Link or None")

    def __str__(self):
        return str(self.getData())


class LinkedList:
    #Minimal Chapter 5 singly linked list.
    def __init__(self):
        self.__first = None

    def getFirst(self):
        return self.__first

    def setFirst(self, link):
        if link is None or isinstance(link, Link):
            self.__first = link
        else:
            raise Exception("First link must be Link or None")

    def isEmpty(self):
        return self.getFirst() is None

    def insert(self, datum):
        #Insert new datum at front of list (used to build factor lists).
        new_link = Link(datum, self.getFirst())
        self.setFirst(new_link)

    def __iter__(self):
        #Allow iteration: for d in linked_list.
        link = self.getFirst()
        while link is not None:
            yield link.getData()
            link = link.getNext()

    def __str__(self):
        pieces = [str(d) for d in self]
        return "[" + " > ".join(pieces) + "]"


def factor(x, lowest=2):
    """
    Recursive factor() per Problem 6.2:

    - Base cases:
        * x == 0  → special case of 0               # “special cases of 0 and 1”
        * x == 1  → special case of 1
        * x == -1 → handle negative 1

    - Negative integers:
        * Prepend -1 then factor(-x, lowest)
          # “factors of the positive version but with the factor 1 replaced by -1”

    - Prime detection:
        * If lowest*lowest > x, x is prime → insert(x)
          # “only the factors between 2 and the square root of x need to be tested”

    - Divide step:
        * If x % lowest == 0:
            -- Recursively factor(x//lowest, lowest)
            -- Then insert(lowest)
          # “If it does, then add lowest to the factors of x divided by lowest.”

    - Skip step:
        * Else → factor(x, lowest+1)
          # “If lowest doesn't evenly divide x, then look for factors with the next higher possible factor.”

    - A factor can appear > once in final list.
    """
    result = LinkedList()

    # ---- Base cases of 0, 1, and -1 ----
    if x == 0:
        result.insert(0)
        return result
    if x == 1:
        result.insert(1)
        return result
    if x == -1:
        result.insert(-1)
        return result

    # ---- Negative integer handling ----
    if x < 0:
        # factors of positive version but with factor 1 replaced by −1
        result.insert(-1)
        tail = factor(-x, lowest)
        for d in tail:          # traverse the positive factors
            result.insert(d)     # prepend so final order is correct
        return result

    # ---- Prime detection (lowest > sqrt(x)) ----
    if lowest * lowest > x:
        result.insert(x)  # x is prime, minimum set of primes includes x itself
        return result

    # ---- Divide step (x % lowest == 0) ----
    if x % lowest == 0:
        tail = factor(x // lowest, lowest)
        for d in tail:      # build up factors of (x//lowest)
            result.insert(d)
        result.insert(lowest)
        return result

    # ---- Skip to next higher integer factor ----
    return factor(x, lowest + 1)


# ---------------- Demo / Output ----------------
if __name__ == "__main__":
    # Test special cases of 0 and 1
    print("factor(0)  # special case 0 →", factor(0))
    print("factor(1)  # special case 1 →", factor(1))

    # Test prime numbers (minimum set of prime factors is itself)
    print("factor(2)  # prime number (evenly divisible only by 1 and itself) →", factor(2))
    print("factor(13) # prime number →", factor(13))

    # Test compound (nonprime) numbers
    print("factor(12) # factors of 12 → minimum set of primes →", factor(12))
    print("factor(60) # factors of 60 →", factor(60))

    # Test negative integer handling
    print("factor(-45) # negative integer → factors of 45 with -1 replacing 1 →", factor(-45))

factor(0)  # special case 0 → [0]
factor(1)  # special case 1 → [1]
factor(2)  # prime number (evenly divisible only by 1 and itself) → [2]
factor(13) # prime number → [13]
factor(12) # factors of 12 → minimum set of primes → [2 > 3 > 2]
factor(60) # factors of 60 → [2 > 3 > 5 > 2]
factor(-45) # negative integer → factors of 45 with -1 replacing 1 → [3 > 5 > 3 > -1]


In [12]:
"""
Problem 6.3:
Implement the recursive approach to raising a number to a power,
as described in the “Raising a Number to a Power” section near the end
of this chapter. Write the recursive power() function. For extra utility,
make use of this transformation to handle negative integer exponents:
    x^y = 1 / x^(-y) when y < 0

Test your function on several combinations including positive and
negative integers and the special cases where the exponent is 0 and 1
(x^0 = 1 and x^1 = x).
"""

def power(x, y):
    """
    Recursive power() per Problem 6.3:

    - Special cases:
        * exponent == 0  → return 1          # “x^0 = 1”
        * exponent == 1  → return x          # “x^1 = x”
    - Negative exponent:
        * y < 0 → use 1 / x^(-y)             # “x^y = 1 / x^-y when y < 0”
    - Recursive (y > 1):
        * x^y = x * x^(y-1)
    """
    # ---- Base cases ----
    if y == 0:
        # “x^0 = 1”
        return 1
    if y == 1:
        # “x^1 = x”
        return x

    # ---- Handle negative integer exponents ----
    if y < 0:
        # “x^y = 1 / x^-y when y < 0”
        return 1 / power(x, -y)

    # ---- Recursive step for positive y > 1 ----
    # “x^y = x * x^(y-1)”
    return x * power(x, y - 1)


# ---------------- Demo / Output ----------------
if __name__ == "__main__":
    # Positive exponent tests
    print("power(2, 0)   # special case x^0=1  →", power(2, 0))
    print("power(5, 1)   # special case x^1=x  →", power(5, 1))
    print("power(3, 3)   # 3^3 = 27            →", power(3, 3))

    # Larger positive exponent
    print("power(2, 10)  # 2^10 = 1024         →", power(2, 10))

    # Negative exponent tests
    print("power(2, -1)  # 2^-1 = 1/2          →", power(2, -1))
    print("power(5, -3)  # 5^-3 = 1/125        →", power(5, -3))

    # Mixed sign
    print("power(-2, 3)  # (-2)^3 = -8         →", power(-2, 3))
    print("power(-2, 4)  # (-2)^4 = 16         →", power(-2, 4))

power(2, 0)   # special case x^0=1  → 1
power(5, 1)   # special case x^1=x  → 5
power(3, 3)   # 3^3 = 27            → 27
power(2, 10)  # 2^10 = 1024         → 1024
power(2, -1)  # 2^-1 = 1/2          → 0.5
power(5, -3)  # 5^-3 = 1/125        → 0.008
power(-2, 3)  # (-2)^3 = -8         → -8
power(-2, 4)  # (-2)^4 = 16         → 16


In [16]:
# %%
"""
Problem 6.4:
Write a program that solves the knapsack problem for an arbitrary
knapsack capacity and series of weights. Assume the weights are stored
in a list.

The recursive knapsack() function should:

1. Base cases:
   * target == 0           → print the current list of selected weights
   * index >= len(weights) → no more items, stop
   * target < 0            → overshot, stop

2. Recursive steps:
   a. Include weights[index]:
      - Copy current list into new_list
      - new_list.insert(weights[index])
      - Recurse with (target - weights[index], index+1, new_list)
   b. Exclude weights[index]:
      - Recurse with (target, index+1, current)

Use Chapter 5’s LinkedList (with __iter__ to traverse).
"""

def knapsack(target, weights, index=0, current=None):
    """
    Recursive knapsack() per Problem 6.4:

    - Base cases:
        * target == 0           → print current list (found solution)
        * index >= len(weights) → ran out of items, stop
        * target < 0            → too heavy, stop

    - Recursive steps:
        1) Include weights[index]:
           - Clone `current` into `new_list`
           - new_list.insert(weights[index])
           - knapsack(target - weights[index], weights, index+1, new_list)
        2) Exclude weights[index]:
           - knapsack(target, weights, index+1, current)
    """
    # ---- Initialize current list on first call ----
    if current is None:
        current = LinkedList()

    # ---- Base cases ----
    if target == 0:
        print(current)
        return
    if index >= len(weights) or target < 0:
        return

    # ---- Include this weight ----
    # clone `current` to preserve it for the exclude branch
    new_list = LinkedList()
    for d in reversed(list(current)):  # preserve insertion order
        new_list.insert(d)
    new_list.insert(weights[index])
    knapsack(target - weights[index], weights, index + 1, new_list)

    # ---- Exclude this weight ----
    knapsack(target, weights, index + 1, current)


# ---------------- Demo / Output ----------------
if __name__ == "__main__":
    W = [11, 8, 7, 6, 5]
    T = 20
    print(f"knapsack({T}, {W}) →")
    knapsack(T, W)

knapsack(20, [11, 8, 7, 6, 5]) →
[5 > 7 > 8]


In [20]:
import random
"""
Problem 7.1:
Extend the ShellSort.py module of Listing 7-1 to include the two other
interval sequences described in the text, namely
            1. dividing the interval by 2.2 and truncating, and
            2. Flamig's formula.
For both methods the first interval, h, should be calculated as if the
preceding value of h was the number of array elements. Set up a program
that creates an input array and passes copies of that array to the three
Shellsort methods (Knuth's sequence, divide by 2.2, and Flamig's sequence).
Compare the number of shifts performed on (a) an array that is already sorted,
(b) an array that inversely sorted, and (c) a random ordering. Use a loop to
perform the test with arrays of length 95 to 100 elements. Is there a clear
method that performs fewer shifts?
"""
# Minimal Array class
class Array:
    def __init__(self, size):
        self._a = [None] * size
        self._n = 0

    def __len__(self):
        return self._n

    def insert(self, item):
        self._a[self._n] = item
        self._n += 1

    def get(self, i):
        return self._a[i] if 0 <= i < self._n else None

    def set(self, i, val):
        if 0 <= i < self._n:
            self._a[i] = val

    def swap(self, i, j):
        if 0 <= i < self._n and 0 <= j < self._n:
            self._a[i], self._a[j] = self._a[j], self._a[i]

    def ShellSort(self):
        """Knuth's 3*h+1 sequence."""
        h = 1
        while 3 * h + 1 < len(self):
            h = 3 * h + 1
        shifts = 0
        while h > 0:
            for outer in range(h, len(self)):
                temp = self.get(outer)
                inner = outer
                while inner >= h and temp < self.get(inner - h):
                    self.set(inner, self.get(inner - h))
                    inner -= h
                    shifts += 1
                if inner < outer:
                    self.set(inner, temp)
                    shifts += 1
            h = (h - 1) // 3
        return shifts


class ExtendedShellArray(Array):
    """Adds two more gap sequences alongside Knuth's."""

    def ShellSort_Knuth(self):
        return self.ShellSort()

    def ShellSort_div2_2(self):
        """Divide-by-2.2 sequence, dropping to zero when done."""
        shifts = 0
        h = int(len(self) / 2.2)
        while h > 0:
            for outer in range(h, len(self)):
                temp = self.get(outer)
                inner = outer
                while inner >= h and temp < self.get(inner - h):
                    self.set(inner, self.get(inner - h))
                    inner -= h
                    shifts += 1
                if inner < outer:
                    self.set(inner, temp)
                    shifts += 1
            h = int(h / 2.2)
        return shifts

    def ShellSort_Flamig(self):
        """Flamig's formula: next h = (5*h-1)//11, dropping to zero below 5."""
        shifts = 0
        N = len(self)
        h = 1 if N < 5 else (5 * N - 1) // 11
        while h > 0:
            for outer in range(h, len(self)):
                temp = self.get(outer)
                inner = outer
                while inner >= h and temp < self.get(inner - h):
                    self.set(inner, self.get(inner - h))
                    inner -= h
                    shifts += 1
                if inner < outer:
                    self.set(inner, temp)
                    shifts += 1
            h = (5 * h - 1) // 11 if h >= 5 else 0
        return shifts


# ---------------- Demo / Comparison ----------------

methods = [
    ("Knuth",  ExtendedShellArray.ShellSort_Knuth),
    ("2.2",    ExtendedShellArray.ShellSort_div2_2),
    ("Flamig", ExtendedShellArray.ShellSort_Flamig),
]

for n in range(95, 101):
    print(f"\nArray length = {n}")
    sorted_base   = list(range(n))
    reversed_base = list(reversed(sorted_base))
    random_base   = sorted_base[:]
    random.shuffle(random_base)

    for label, base in [("sorted", sorted_base),
                        ("reversed", reversed_base),
                        ("random", random_base)]:
        print(f"  {label:>8} →", end=" ")
        results = []
        for name, func in methods:
            arr = ExtendedShellArray(n)
            for x in base:
                arr.insert(x)
            results.append(f"{name}={func(arr)}")
        print(", ".join(results))


Array length = 95
    sorted → Knuth=0, 2.2=0, Flamig=0
  reversed → Knuth=481, 2.2=386, Flamig=338
    random → Knuth=646, 2.2=610, Flamig=452

Array length = 96
    sorted → Knuth=0, 2.2=0, Flamig=0
  reversed → Knuth=424, 2.2=444, Flamig=306
    random → Knuth=628, 2.2=563, Flamig=430

Array length = 97
    sorted → Knuth=0, 2.2=0, Flamig=0
  reversed → Knuth=418, 2.2=400, Flamig=372
    random → Knuth=575, 2.2=542, Flamig=410

Array length = 98
    sorted → Knuth=0, 2.2=0, Flamig=0
  reversed → Knuth=422, 2.2=410, Flamig=340
    random → Knuth=653, 2.2=544, Flamig=375

Array length = 99
    sorted → Knuth=0, 2.2=0, Flamig=0
  reversed → Knuth=431, 2.2=409, Flamig=311
    random → Knuth=632, 2.2=509, Flamig=412

Array length = 100
    sorted → Knuth=0, 2.2=0, Flamig=0
  reversed → Knuth=420, 2.2=426, Flamig=294
    random → Knuth=660, 2.2=631, Flamig=448


In [ ]:
# Problem 7.2
"""
7.2 Modify the quicksort() and helper methods in Listings 7-7 and 7-8 to
count the number of item copies and key comparisons they make during
a sort and return the counts to the caller (using Python’s ability to return
multiple values). For the copies, a swap counts as 3 copies. For the
comparisons, only count comparisons of key values (not array indices).
Use the modified method to show how many copies and comparisons
are made when sorting (a) a forward-sorted array of 50 items, (b) a
reverse-sorted array of 50 items, (c) a constant value for all 50 items,
and (d) an array of 50 random items. For each of those array styles,
show the counts when short is 3, 7, and 11 to see the effect of using
insertion sort for the shorter subarrays. Use the same random array for
all variations of the short parameter to make a fair comparison.
"""

import random

class CountingArray:
    #A wrapper around a Python list that counts copies and comparisons.
    def __init__(self, items):
        # initialize array and counters
        self.a = list(items)
        self.copy_count = 0       # total item copies
        self.comp_count = 0       # total key comparisons

    def get(self, i):
        # no count for reading
        return self.a[i]

    def set(self, i, v):
        # setting an element counts as 1 copy
        self.a[i] = v
        self.copy_count += 1

    def swap(self, i, j):
        # swapping two elements counts as 3 copies
        self.a[i], self.a[j] = self.a[j], self.a[i]
        self.copy_count += 3

    def insertionSort(self, lo, hi):
        # for subarrays of length <= short: insertion sort
        for outer in range(lo + 1, hi + 1):
            temp = self.a[outer]            # no copy counted for get
            inner = outer
            # compare key values and shift as needed
            while inner > lo:
                self.comp_count += 1        # count comparison
                if temp < self.a[inner - 1]:
                    self.set(inner, self.a[inner - 1])  # shift: 1 copy
                    inner -= 1
                else:
                    break
            # place temp in its correct spot
            self.set(inner, temp)          # insertion: 1 copy

    def medianOfThree(self, lo, hi):
        # choose pivot by median-of-three and place median at hi
        mid = (lo + hi) // 2
        # compare lo vs mid
        self.comp_count += 1
        if self.a[lo] > self.a[mid]:
            self.swap(lo, mid)
        # compare lo vs hi
        self.comp_count += 1
        if self.a[lo] > self.a[hi]:
            self.swap(lo, hi)
        # compare hi vs mid
        self.comp_count += 1
        if self.a[hi] > self.a[mid]:
            self.swap(hi, mid)
        return self.a[hi]                 # pivot value now at hi

    def __part(self, pivot, lo, hi):
        # partition subarray [lo..hi] around pivot
        while lo <= hi:
            # move lo right until self.a[lo] >= pivot
            while True:
                self.comp_count += 1      # count comparison
                if self.a[lo] < pivot:
                    lo += 1
                else:
                    break
            # move hi left until self.a[hi] <= pivot
            while True:
                self.comp_count += 1      # count comparison
                if pivot < self.a[hi]:
                    hi -= 1
                else:
                    break
            if lo >= hi:
                return lo
            # swap out-of-place elements
            self.swap(lo, hi)
            lo += 1; hi -= 1
        return lo

    def quicksort(self, lo=0, hi=None, short=3):
        # reset counters on top-level call
        if hi is None:
            hi = len(self.a) - 1
            self.copy_count = 0
            self.comp_count = 0

        short = max(3, short)             # enforce minimum cutoff

        # if subarray is short, use insertion sort
        if hi - lo + 1 <= short:
            self.insertionSort(lo, hi)
            return self.copy_count, self.comp_count

        # select pivot via median-of-three
        pivot = self.medianOfThree(lo, hi)
        # partition excluding pivot at hi
        hipart = self.__part(pivot, lo + 1, hi - 1)
        # place pivot in its final position
        if hipart < hi:
            self.swap(hipart, hi)

        # recursively sort left and right partitions
        self.quicksort(lo, hipart - 1, short)
        self.quicksort(hipart + 1, hi, short)
        return self.copy_count, self.comp_count

# --- test harness for (a)-(d) with short = 3,7,11 ---

# (a) forward-sorted, (b) reverse-sorted, (c) constant, (d) random
forward = list(range(50))
reverse_ = forward[::-1]
constant = [42] * 50
random.seed(0)
random_arr = [random.randint(0, 100) for _ in range(50)]

datasets = {
    'Forward-Sorted': forward,
    'Reverse-Sorted': reverse_,
    'Constant': constant,
    'Random': random_arr
}
short_values = [3, 7, 11]

# display results
print(f"{'Dataset':<15}{'Short':<6}{'Copies':<10}{'Comparisons'}")
for name, data in datasets.items():
    for s in short_values:
        arr = CountingArray(data)
        copies, comps = arr.quicksort(short=s)
        print(f"{name:<15}{s:<6}{copies:<10}{comps}")

Dataset        Short Copies    Comparisons
Forward-Sorted 3     109       253
Forward-Sorted 7     77        202
Forward-Sorted 11    73        190
Reverse-Sorted 3     184       251
Reverse-Sorted 7     152       200
Reverse-Sorted 11    148       188
Constant       3     292       230
Constant       7     251       190
Constant       11    238       179
Random         3     261       249
Random         7     225       228
Random         11    225       222
